In [1]:
%pwd

'/media/wout/Data/PhD/pp/code/experiments/kfold'

In [2]:
%cd ../..

/media/wout/Data/PhD/pp/code


In [77]:
import tensorflow as tf
import io
from PIL import Image
import ipywidgets as W
from IPython.display import display
from tensorflow.keras import Model
from tensorflow import keras

import experiment as exp

import shared.util.dataset_extra as ds_extra
from tensorflow.keras.preprocessing.image import array_to_img as KerasImg

In [84]:
flags = exp.Flags(dataset="mnist")
config = exp.CONFIG[flags.dataset]
ass_ds = tf.data.experimental.load(exp.CHECKPOINT_DIR_DATASET(flags.dataset))
assessor, _, _ = exp.try_restore_model(config.assessor(), exp.CHECKPOINT_DIR_ASSESSOR(flags.dataset))

Using model checkpoint assets/models/kfold/mnist_assessor/ckpt-15


In [64]:
def PIL_to_bytes(img):
    # https://stackoverflow.com/questions/33101935/convert-pil-image-to-byte-array
    img_bytes = io.BytesIO()
    img.save(img_bytes, format='png')
    return img_bytes.getvalue()

In [94]:
def Instance(inst, model: Model):
    """
    Accept a FeatureDict TF Dataset instance, and return a widget that renders it.
    { "image": img, "label": ..., "prediction": ..., "loss": ... }
    """
    x = inst['image']
    base_loss_act = inst['loss']
    base_loss_pred = model(x.reshape((1) + x.shape))
    ass_loss = keras.losses.mean_squared_error(base_loss_act, base_loss_pred)
    
    return W.HBox([
        W.Image(value=PIL_to_bytes(KerasImg(inst["image"]).resize((100, 100)))),
        W.VBox([
            W.Label(value=f"Index: {inst['index']}"),
            W.Label(value=f"Label (actual): {inst['label']}"),
            W.Label(value=f"Label (predic): {inst['prediction']}"),
            W.Label(value=f"Base Loss (actual): {base_loss_act}"),
            W.Label(value=f"Base Loss (predic): {base_loss_pred}"),
            W.Label(value=f"Assessor Loss: {ass_loss}")
        ])
    ])

# Wtf is instance 3240?
inst = ds_extra.peek(
#     ass_ds.skip(6)
    ass_ds.filter(lambda inst: inst['loss'] > 0.0).skip(100)
)
display(Instance(inst, assessor))

In [8]:
w.value = 10